In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import os.path
import torch
import torch.utils.data as data
import skimage.io as io
from skimage import color
import cv2
from google.colab import drive
### only for reference
import time
import torch
import random
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt

In [ ]:
learning_rates = [3e-1, 3e-2, 3e-3, 3e-4]
num_epochs = 25

In [ ]:
### U-Net
def conv_block(in_channels, out_channels):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, 3, padding=1),
        nn.ReLU(inplace=True),
        nn.Conv2d(out_channels, out_channels, 3, padding=1),
        nn.ReLU(inplace=True)
    )   


class UNet(nn.Module):

    def __init__(self, n_class):
        super().__init__()
                
        self.conv_down_1 = conv_block(3, 8)
        self.conv_down_2 = conv_block(8, 16)
        self.conv_down_3 = conv_block(16, 32)
        self.conv_down_4 = conv_block(32, 64)        

        self.maxpool = nn.MaxPool2d(2)
        self.upsample = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)        
        
        self.conv_up_3 = conv_block(32 + 64, 32)
        self.conv_up_2 = conv_block(32 + 16, 16)
        self.conv_up_1 = conv_block(16 + 8, 8)
        
        self.conv_last = nn.Conv2d(8, n_class, 1)
        
        
    def forward(self, x):
        conv1 = self.conv_down_1(x)
        x = self.maxpool(conv1)

        conv2 = self.conv_down_2(x)
        x = self.maxpool(conv2)
        
        conv3 = self.conv_down_3(x)
        x = self.maxpool(conv3)   
        
        x = self.conv_down_4(x)
        
        x = self.upsample(x)        
        x = torch.cat([x, conv3], dim=1)
        
        x = self.conv_up_3(x)
        x = self.upsample(x)        
        x = torch.cat([x, conv2], dim=1)       

        x = self.conv_up_2(x)
        x = self.upsample(x)        
        x = torch.cat([x, conv1], dim=1)   
        
        x = self.conv_up_1(x)
        
        out = self.conv_last(x)
        
        return out

In [ ]:
### Remember to customize your own path (This is for anchor and positive images)
def create_images(img_h,img_w,num_training_samples):
  drive.mount('/content/drive')
  #num_images = 200
  #num_training_samples = 10 #(more training_samples is better)
  #img_h = 256 #Set uniform height for images(please change the height)
  #img_w = 256 #Set uniform width for images(please change the width)
  path_L_train = '/content/drive/MyDrive/EECS 545 Project/data_scene_flow/training/image_2'

  train_indices = range(0,num_training_samples) # 80% of the training data is used for training

  img_ext = '.png'

  #Training Data (This is for anchor and positive images)

  X_L_train = np.zeros((num_training_samples, img_h, img_w, 3),dtype=np.float64)
  y_L_train = np.zeros((num_training_samples, img_h, img_w, 3),dtype=np.float64)

  for i in train_indices:
    file_idx = '%.6d' % i

    path_Lx = os.path.join(path_L_train, file_idx + "_10" + img_ext)
    read_Lx = io.imread(path_Lx)
    im_Lx = read_Lx #color.rgb2gray(read_Lx)
    path_Ly = os.path.join(path_L_train, file_idx + "_11" + img_ext)
    read_Ly = io.imread(path_Ly)
    im_Ly = read_Ly #color.rgb2gray(read_Ly)


    im_Lx_resized = cv2.resize(src=im_Lx, dsize=(img_w, img_h))
    im_Ly_resized = cv2.resize(src=im_Ly, dsize=(img_w, img_h))

    #print("im_Lx_resized shape:",np.shape(im_Lx_resized))
    #print("X_L shape:",np.shape(X_L_train))
    #print("X_L[i] shape:",np.shape(X_L_train[i]))

    X_L_train[i] = im_Lx_resized
    y_L_train[i] = im_Ly_resized



  ### loading and processing data

  X_train = X_L_train.transpose(0, 3, 1, 2)
  y_train = y_L_train.transpose(0, 3, 1, 2)
  X_train = torch.tensor(X_train, dtype=torch.float)
  y_train = torch.tensor(y_train, dtype=torch.float)

  ### assign to anchor and positve images:
  anchor_imgs = X_train
  positive_imgs = y_train

  return anchor_imgs, positive_imgs

In [ ]:
import torch.nn.functional as F 
def train_model(model, criterion, optimizer, num_epochs, anchor_imgs, positive_imgs):
    model.train() 

    total_loss =[]
    for epoch in tqdm(range(num_epochs), desc="Epochs"):
        running_loss = []
        for i in range(anchor_imgs.shape[0]):
            anchor_img = anchor_imgs[i].to(device)
            anchor_img = torch.unsqueeze(anchor_img, 0)
            positive_img = positive_imgs[i].to(device)
            positive_img = torch.unsqueeze(positive_img, 0)
            
            optimizer.zero_grad()
            anchor_out = model(anchor_img) 

            # Adding a sigmoid so the loss is better (mapping to range [0, 1])
            anchor_out = torch.sigmoid(anchor_out)
            positive_img = positive_img / 256.0

            loss = criterion(anchor_out, positive_img)
            loss.backward()
            optimizer.step()
            
            running_loss.append(loss.cpu().detach().numpy())
        total_loss.append(running_loss[0])
        print("Epoch: {}/{} - Loss: {:.8f}".format(epoch+1, num_epochs, np.mean(running_loss)))
    return model,total_loss

In [ ]:
anchor_imgs,positive_imgs = create_images(224,224,200)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
anchor_imgs.shape
positive_imgs.shape

torch.Size([200, 3, 224, 224])

In [ ]:
# Try different learning rates
i = 0
for learning_rate in learning_rates:
  print("Trying learning rate", learning_rate)
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  model = UNet(3)
  model = model.to(device)
  optimizer = optim.Adam(model.parameters(), lr=learning_rate)
  criterion = torch.nn.MSELoss()
  model,loss = train_model(model, criterion , optimizer, num_epochs, anchor_imgs, positive_imgs)
  torch.save(model.state_dict(), '/content/drive/MyDrive/EECS 545 Project/saved_model/JoeyModel' + str(i) + '.pkl')
  i += 1

Trying learning rate 0.3


Epoch: 1/25 - Loss: 0.10526404
Epoch: 2/25 - Loss: 0.10510942
Epoch: 3/25 - Loss: 0.10528734
Epoch: 4/25 - Loss: 0.10538264
Epoch: 5/25 - Loss: 0.10544755
Epoch: 6/25 - Loss: 0.10549725
Epoch: 7/25 - Loss: 0.10553796
Epoch: 8/25 - Loss: 0.10557257
Epoch: 9/25 - Loss: 0.10560352
Epoch: 10/25 - Loss: 0.10563090
Epoch: 11/25 - Loss: 0.10565630
Epoch: 12/25 - Loss: 0.10568012
Epoch: 13/25 - Loss: 0.10570212
Epoch: 14/25 - Loss: 0.10572283
Epoch: 15/25 - Loss: 0.10574282
Epoch: 16/25 - Loss: 0.10576193
Epoch: 17/25 - Loss: 0.10578005
Epoch: 18/25 - Loss: 0.10579729
Epoch: 19/25 - Loss: 0.10581404
Epoch: 20/25 - Loss: 0.10583034
Epoch: 21/25 - Loss: 0.10584594
Epoch: 22/25 - Loss: 0.10586137
Epoch: 23/25 - Loss: 0.10587595
Epoch: 24/25 - Loss: 0.10589047
Epoch: 25/25 - Loss: 0.10590416

Trying learning rate 0.03


Epoch: 1/25 - Loss: 0.32496929
Epoch: 2/25 - Loss: 0.33002162
Epoch: 3/25 - Loss: 0.33002162
Epoch: 4/25 - Loss: 0.33002162
Epoch: 5/25 - Loss: 0.33002162
Epoch: 6/25 - Loss: 0.33002162
Epoch: 7/25 - Loss: 0.33002162
Epoch: 8/25 - Loss: 0.33002162
Epoch: 9/25 - Loss: 0.33002162
Epoch: 10/25 - Loss: 0.33002162
Epoch: 11/25 - Loss: 0.33002162
Epoch: 12/25 - Loss: 0.33002162
Epoch: 13/25 - Loss: 0.33002162
Epoch: 14/25 - Loss: 0.33002162
Epoch: 15/25 - Loss: 0.33002162
Epoch: 16/25 - Loss: 0.33002162
Epoch: 17/25 - Loss: 0.33002162
Epoch: 18/25 - Loss: 0.33002162
Epoch: 19/25 - Loss: 0.33002162
Epoch: 20/25 - Loss: 0.33002162
Epoch: 21/25 - Loss: 0.33002162
Epoch: 22/25 - Loss: 0.33002162
Epoch: 23/25 - Loss: 0.33002162
Epoch: 24/25 - Loss: 0.33002162
Epoch: 25/25 - Loss: 0.33002162

Trying learning rate 0.003


Epoch: 1/25 - Loss: 0.04296434
Epoch: 2/25 - Loss: 0.02636679
Epoch: 3/25 - Loss: 0.02648315
Epoch: 4/25 - Loss: 0.02596666
Epoch: 5/25 - Loss: 0.02542559
Epoch: 6/25 - Loss: 0.02586889
Epoch: 7/25 - Loss: 0.02584630
Epoch: 8/25 - Loss: 0.02886955
Epoch: 9/25 - Loss: 0.02571657
Epoch: 10/25 - Loss: 0.02544570
Epoch: 11/25 - Loss: 0.02533673
Epoch: 12/25 - Loss: 0.02531551
Epoch: 13/25 - Loss: 0.02522973
Epoch: 14/25 - Loss: 0.02510229
Epoch: 15/25 - Loss: 0.02513428
Epoch: 16/25 - Loss: 0.02507945
Epoch: 17/25 - Loss: 0.02506251
Epoch: 18/25 - Loss: 0.02499861
Epoch: 19/25 - Loss: 0.02501173
Epoch: 20/25 - Loss: 0.02494493
Epoch: 21/25 - Loss: 0.02491558
Epoch: 22/25 - Loss: 0.02489560
Epoch: 23/25 - Loss: 0.02491162
Epoch: 24/25 - Loss: 0.02490692
Epoch: 25/25 - Loss: 0.02487206

Trying learning rate 0.0003


Epoch: 1/25 - Loss: 0.06209154
Epoch: 2/25 - Loss: 0.02826498
Epoch: 3/25 - Loss: 0.02504151
Epoch: 4/25 - Loss: 0.02371020
Epoch: 5/25 - Loss: 0.02311405
Epoch: 6/25 - Loss: 0.02271251
Epoch: 7/25 - Loss: 0.02247917
Epoch: 8/25 - Loss: 0.02232713
Epoch: 9/25 - Loss: 0.02217372
Epoch: 10/25 - Loss: 0.02207457
Epoch: 11/25 - Loss: 0.02195735
Epoch: 12/25 - Loss: 0.02181842
Epoch: 13/25 - Loss: 0.02157948
Epoch: 14/25 - Loss: 0.02146880
Epoch: 15/25 - Loss: 0.02129220
Epoch: 16/25 - Loss: 0.02114351
Epoch: 17/25 - Loss: 0.02099151
Epoch: 18/25 - Loss: 0.02085946
Epoch: 19/25 - Loss: 0.02075095
Epoch: 20/25 - Loss: 0.02063238
Epoch: 21/25 - Loss: 0.02053245
Epoch: 22/25 - Loss: 0.02035996
Epoch: 23/25 - Loss: 0.02025180
Epoch: 24/25 - Loss: 0.02015642
Epoch: 25/25 - Loss: 0.01997876



In [ ]:
loss = np.float32(loss)
num = 20
epochs = np.linspace(1,num,num)

plt.plot(epochs,loss)
plt.title('Training loss vs epochs')
plt.xlabel('loss')
plt.ylabel('epochs')